In [1]:
import pandas as pd
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

### RUNS.CSV

In [ ]:
df_runs = pd.read_csv("../data/raw/runs.csv")
df_runs.head(15)

In [ ]:
df_runs.info()

In [5]:
df_for_prediction = df_runs.drop(columns =['finish_time','horse_no'])
df_for_prediction = df_for_prediction.drop(columns =['position_sec1','position_sec2','position_sec3','position_sec4','position_sec5','position_sec6'])
df_for_prediction = df_for_prediction.drop(columns =['behind_sec1','behind_sec2','behind_sec3','behind_sec4','behind_sec5','behind_sec6'])
df_for_prediction = df_for_prediction.drop(columns =['time1','time2','time3','time4','time5','time6'])


In [ ]:
df_for_prediction

In [ ]:
df_for_prediction.query("horse_gear == '--'").count()

In [ ]:
df_for_prediction.info()

### RACES.CSV

In [ ]:
df_races = pd.read_csv("../data/raw/races.csv")

df_races = df_races.drop(columns =['sec_time1','sec_time2','sec_time3','sec_time4','sec_time5','sec_time6', 'sec_time7'])
df_races = df_races.drop(columns =['time1','time2','time3','time4','time5','time6', 'time7'])

df_races

In [ ]:
df_races['date'] = pd.to_datetime(df_races['date'])
df_races.info()

In [ ]:
print(df_races['date'].min())
print(df_races['date'].max())

In [ ]:
print(df_races['prize'].min())
print(df_races['prize'].max())

In [ ]:
df_races.tail()

In [ ]:
df_races['surface'].unique()

In [ ]:
df_races['race_no'].unique()

Faire un seul dataframe possédant une ligne par produit cartésien cheval x course 

In [ ]:
df = pd.merge(df_races,df_for_prediction, on='race_id', how='inner')

df.head()

race_id : put in index
date : remove but keep rows in order
venue : one hot encoding
race_no : remove
config : ? remove ?
surface : one hot encoding
distance : one hot encoding
going : ? keep ?
horse_ratings : always the same ?
prize : remove (we will use that information in the process but not in the prediction)
race_class : what is it ?
place_combination1,2,3 : remove (info for the y axis)
place_combination4 : remove
place_dividend1,2,3 : remove (info for the y axis) > make 2 analysis after, based on if the horse is placed or not based on statistics
place_dividend4 : remove
win_comlbination1 : remove (info for the y axis)
win_dividend1 : remove (info for the y axis)
win_combination2 : remove (cas particulier)
win_dividend2 : remove (cas particulier)
horse_id : c'est notre y
result : c'est notre y ? (faire une métrique genre average ranking over the last 30 races)
won : c'est notre y
lengths_behind : keep but metric (average lenghts behind over the last 30 races)
horse_age : keep
horse_country : one hot encoding but why not remove
horse_rating : always the same ?
horse_gear: always blank ?
declared_weight : keep
actual_weight : keep
draw : ?
win_odds : keep
place_odds : keep 
trainer_id : keep one hot
jockey_id : keep one hot

In [ ]:
df['distance'].unique()

In [ ]:
mask = df[df['race_id'] == 0]

mask

In [ ]:
race_counts = df['race_id'].value_counts()
average_rows_per_race = race_counts.mean()
print("Average number of rows per unique race_id:", average_rows_per_race)

garder cette formulation en tête

In [ ]:
df.query("race_id == 0 and venue == 'ST'")

un cheval participe t'il à plusieurs courses au sein de la même journée

In [ ]:
duplicates = df.groupby(['date', 'horse_id']).size().reset_index(name='count')

# Filter rows where count is greater than 1
duplicates = duplicates[duplicates['count'] > 1]

# Display result
print(duplicates)

non

In [ ]:
print(df['surface'].nunique())
print(df['distance'].nunique())
print(df['going'].nunique())
print(df['horse_country'].nunique())
print(df['horse_type'].nunique())
print(df['horse_gear'].nunique())
print(df['trainer_id'].nunique())
print(df['jockey_id'].nunique())
print(df['horse_id'].nunique())


going c l'état de la piste

In [ ]:
df['going'].unique()

In [24]:
df = df.drop(columns=['date','race_no','horse_ratings','prize','result',
                                         'place_combination1','place_combination2','place_combination3','place_combination4','place_dividend1','place_dividend2','place_dividend3','place_dividend4',
                                         'win_combination1','win_dividend1','win_combination2','win_dividend2',
                                         'lengths_behind','horse_gear','trainer_id','jockey_id'])
# réintégrer trainer_id et jockey_id en categorical

In [ ]:
df.set_index("race_id",inplace=True)
df

In [ ]:
df = pd.get_dummies(df, columns=['venue','config','race_class','surface', 'distance', 'going', 'horse_country', 'horse_type'], dtype=int, drop_first=True)
df


In [ ]:
df['place_odds'].fillna(0, inplace=True)

In [28]:
train_ratio = 0.8
split_index = int(len(df)*train_ratio)

df_train = df.iloc[:split_index]
df_test = df.iloc[split_index:]


In [ ]:
df_test

In [30]:
df_train_original = df_train.copy()
df_test_original = df_test.copy()

y_train = df_train['won']
X_train = df_train.drop(columns=['won','horse_id'])

y_test = df_test['won']
X_test = df_test.drop(columns=['won','horse_id'])

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [33]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,y_train)

y_predict = model.predict(X_test)

In [ ]:
y_predict

In [ ]:
import matplotlib.pyplot as plt
# plt.hist(y_predict, density=True)

import seaborn as sns
sns.kdeplot(y_predict)

merge le y_predict en prenant le max pour chacune des courses pour donner un gagnant puis faire de la classif ? ou faire de la classif d'abord

In [36]:
df = df_test_original.copy()
df['y_predict'] = y_predict
df_stats = df.copy()

df_stats['max_prediction'] = df_stats.groupby(level=0)['y_predict'].transform('max')
df_stats['top_prediction'] = (df_stats['y_predict'] == df_stats['max_prediction']).astype(int)
df_stats.drop(columns=['max_prediction'],inplace=True)


In [ ]:
pd.reset_option('display.max_rows')
df_stats = df_stats[['horse_id','won','y_predict','top_prediction']]
df_stats.head()


In [38]:
df_stats['won'] = df_stats['won'].astype(int)

In [ ]:
df_stats.info()

In [ ]:
#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

df_stats['accuracy'] = ((df_stats['won'] == 1) & (df_stats['top_prediction'] == 1))
df_stats

In [41]:
total_races = df_stats.index.nunique()

correct_prediction_races = df_stats.groupby(df_stats.index)['accuracy'].max().sum()
accuracy = (correct_prediction_races / total_races) * 100


In [ ]:
print(f"Précision basée sur les courses : {accuracy:.2f}%")

Test du refacto

In [ ]:
import pandas as pd

def prepare_predictions(df_test_original, y_predict):
    """
    Crée une copie du DataFrame de test et ajoute les prédictions.
    """
    df = df_test_original.copy()
    df['y_predict'] = y_predict
    return df

def compute_top_prediction(df):
    """
    Détermine le cheval avec la meilleure prédiction par course.
    """
    df['max_prediction'] = df.groupby(level=0)['y_predict'].transform('max')
    df['top_prediction'] = (df['y_predict'] == df['max_prediction']).astype(int)
    df.drop(columns=['max_prediction'], inplace=True)
    return df

def finalize_dataframe(df):
    """
    Réorganise le DataFrame et assure le bon format des données.
    """
    df = df[['horse_id', 'won', 'y_predict', 'top_prediction']]
    df['won'] = df['won'].astype(int)
    return df

def compute_accuracy(df):
    """
    Calcule l'accuracy des prédictions sur les courses.
    """
    df['accuracy'] = ((df['won'] == 1) & (df['top_prediction'] == 1))
    
    total_races = df.index.nunique()
    correct_prediction_races = df.groupby(df.index)['accuracy'].max().sum()
    
    accuracy = (correct_prediction_races / total_races) * 100
    return accuracy

# 📌 Utilisation des fonctions pour comparer avec ta méthode actuelle
df = prepare_predictions(df_test_original, y_predict)
df = compute_top_prediction(df)
df_stats = finalize_dataframe(df)

accuracy = compute_accuracy(df_stats)

print(f"✅ Précision basée sur les courses : {accuracy:.2f}%")


XGBCLASSIFIER

In [ ]:
pip install xgboost

In [45]:
import xgboost
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(X_train,y_train)

y_xgb_predict = xgb.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
# plt.hist(y_predict, density=True)

import seaborn as sns
sns.kdeplot(y_xgb_predict)

In [ ]:
df_stats['y_xgb_predict'] = y_xgb_predict

df_stats['max_xgb_prediction'] = df_stats.groupby(level=0)['y_xgb_predict'].transform('max')
df_stats['top_xgb_prediction'] = (df_stats['y_xgb_predict'] == df_stats['max_xgb_prediction']).astype(int)
df_stats.drop(columns=['max_xgb_prediction'],inplace=True)
df_stats = df_stats[['horse_id','won','y_xgb_predict','top_xgb_prediction']]
df_stats.head()

In [48]:
df_stats['accuracy_xgb'] = ((df_stats['won'] == 1) & (df_stats['top_xgb_prediction'] == 1))
total_races = df_stats.index.nunique()
correct_prediction_races = df_stats.groupby(df_stats.index)['accuracy_xgb'].max().sum()
accuracy_xgb = (correct_prediction_races / total_races) * 100

In [ ]:

print(f"Précision basée sur les courses avec xgboost : {accuracy_xgb:.2f}%")

In [ ]:
df.head()

Stats and Expectancy

In [ ]:
K = 2.5

dfe = df[['horse_id','won','win_odds','place_odds','y_predict']]
dfe['profitable_bet_win'] = (dfe['y_predict']>1/dfe['win_odds'])
dfe['profitable_bet_place'] = ((df['y_predict'] * K).clip(upper=1.0)>1/dfe['place_odds'])
dfe.head(24)

Calculer les profits de base